# Unit test data

This directory contains very small, toy, data sets that are used
for unit tests.

## Object catalog: small_sky

This "object catalog" is 131 randomly generated radec values.

- All radec positions are in the Healpix pixel order 0, pixel 11.
- IDs are integers from 700-831.

The following are imports and paths that are used throughout the notebook.

In [ ]:
import shutil
import tempfile
from pathlib import Path

import lsdb
import pyarrow as pa
import pyarrow.dataset as pds
import pyarrow.parquet as pq
from dask.distributed import Client
from hats.io.file_io import remove_directory

from hats_import import pipeline_with_client, ImportArguments

tmp_path = tempfile.TemporaryDirectory()
tmp_dir = tmp_path.name

hats_import_dir = "."
client = Client(n_workers=1, threads_per_worker=1, local_directory=tmp_dir)

### small_sky

This "object catalog" is 131 randomly generated radec values.

- All radec positions are in the Healpix pixel order 0, pixel 11.
- IDs are integers from 700-831.

This catalog was generated with the following snippet:

In [ ]:
remove_directory("./small_sky_object_catalog")
with tempfile.TemporaryDirectory() as pipeline_tmp:
    args = ImportArguments(
        input_path=Path(hats_import_dir) / "small_sky",
        output_path=".",
        file_reader="csv",
        highest_healpix_order=5,
        output_artifact_name="small_sky_object_catalog",
        tmp_dir=pipeline_tmp,
        addl_hats_properties={"hats_cols_default": ["ra", "dec", "id"]},
    )
    pipeline_with_client(args, client)

## Source catalog: small_sky_source

This "source catalog" is 131 detections at each of the 131 objects
in the "small_sky" catalog. These have a random magnitude, MJD, and
band (selected from ugrizy). The full script that generated the values
can be found [here](https://github.com/delucchi-cmu/hipscripts/blob/main/twiddling/small_sky_source.py)

The catalog was generated with the following snippet, using raw data
from the `hats-import` file.

NB: `pixel_threshold=3000` is set just to make sure that we're generating
a handful of files at various healpix orders.

In [ ]:
remove_directory("./small_sky_source_catalog")
with tempfile.TemporaryDirectory() as pipeline_tmp:
    args = ImportArguments(
        input_path=Path(hats_import_dir) / "small_sky_source",
        output_path=".",
        file_reader="csv",
        ra_column="source_ra",
        dec_column="source_dec",
        catalog_type="source",
        highest_healpix_order=5,
        pixel_threshold=3000,
        drop_empty_siblings=False,
        output_artifact_name="small_sky_source_catalog",
        tmp_dir=pipeline_tmp,
    )
    pipeline_with_client(args, client)

In [ ]:
client.close()
tmp_path.cleanup()

## Nested catalog: small_sky_nested_catalog

In [ ]:
small_sky_object = lsdb.read_hats("small_sky_object_catalog", columns="all")
small_sky_source = lsdb.read_hats("small_sky_source_catalog")
small_sky_nested = small_sky_object.join_nested(
    small_sky_source, left_on="id", right_on="object_id", nested_column_name="lc"
)
lsdb.io.to_hats(
    small_sky_nested,
    base_catalog_path="small_sky_nested_catalog",
    catalog_name="small_sky_nested_catalog",
    histogram_order=5,
    overwrite=True,
)

## Malformed Catalogs: bad_schemas and wrong_files_and_rows

These datasets are designed to fail verification tests.
They are generated by mangling `small_sky_object_catalog`.

In [ ]:
# Load the input data that will be used to generate the malformed catalogs.
input_dataset_path = Path(hats_import_dir) / "small_sky_object_catalog" / "dataset"
input_ds = pds.parquet_dataset(input_dataset_path / "_metadata")

# Unit tests expect the Npix=11 data file
input_frag = next(frag for frag in input_ds.get_fragments() if frag.path.endswith("Npix=11.parquet"))
frag_key = Path(input_frag.path).relative_to(input_dataset_path)
input_tbl = input_frag.to_table()

In [ ]:
def collect_and_write_metadata(output_dataset_path: Path, schema: pa.Schema | None = None) -> None:
    schema = schema or input_tbl.schema
    dataset = pds.dataset(output_dataset_path)
    metadata_collector = []
    for frag in dataset.get_fragments():
        frag.ensure_complete_metadata()
        frag.metadata.set_file_path(str(Path(frag.path).relative_to(output_dataset_path)))
        metadata_collector.append(frag.metadata)
    pq.write_metadata(
        schema=schema, where=output_dataset_path / "_metadata", metadata_collector=metadata_collector
    )

### bad_schemas

This dataset is designed to fail all schema verification tests.

```
bad_schemas/
|- dataset/
    |- _common_metadata.import_truth        # mimics schema provided by user upon import
    |- _common_metadata                     # wrong types
    |- _metadata                            # wrong file-level metadata
    |- Norder=0/Dir=0/
        |- Npix=11.parquet                  # direct copy of input
        |- Npix=11.extra_column.parquet     # extra column
        |- Npix=11.missing_column.parquet   # missing column
        |- Npix=11.wrong_dtypes.parquet     # wrong types
        |- Npix=11.wrong_metadata.parquet   # wrong metadata
```

In [ ]:
output_dataset_path = Path(".") / "bad_schemas" / "dataset"
remove_directory(output_dataset_path)

# Existing files may result in unexpected metadata output.
if output_dataset_path.parent.exists() and any(output_dataset_path.parent.iterdir()):
    raise FileExistsError("bad_schemas directory exists and is not empty. Remove it and try again.")

# We will create the following files using input_frag
ffrag_out = output_dataset_path / frag_key
fextra_col = ffrag_out.with_suffix(".extra_column.parquet")
fmissing_col = ffrag_out.with_suffix(".missing_column.parquet")
fwrong_types = ffrag_out.with_suffix(".wrong_dtypes.parquet")
fwrong_metadata = ffrag_out.with_suffix(".wrong_metadata.parquet")

ffrag_out.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
# Make a direct copy of input_frag for all files that will be recorded in the _metadata file
for file_out in [ffrag_out, fmissing_col, fextra_col, fwrong_types]:
    shutil.copy(input_frag.path, file_out)

# Write a _metadata that has the correct schema except for file-level metadata
metadata = input_tbl.schema.metadata or {}
metadata.update({b"extra key": b"extra value"})
collect_and_write_metadata(output_dataset_path, schema=input_tbl.schema.with_metadata(metadata))

In [ ]:
# Write new data files using incorrect schemas.

# Drop a column
pq.write_table(input_tbl.drop_columns("dec_error"), fmissing_col)

# Add an extra column
extra_col = pa.array(range(len(input_tbl)))
extra_col_tbl = input_tbl.add_column(5, pa.field("extra", pa.int64()), extra_col)
pq.write_table(extra_col_tbl, fextra_col)

# Mangle file-level metadata
wrong_metadata = {"bad key": "bad value"}
pq.write_table(input_tbl.replace_schema_metadata(wrong_metadata), fwrong_metadata)

# Change some types
wrong_dtypes_fields = [
    fld if not fld.name.startswith("ra") else fld.with_type(pa.float16()) for fld in input_tbl.schema
]
wrong_dtypes_schema = pa.schema(wrong_dtypes_fields, metadata=input_tbl.schema.metadata)
pq.write_table(input_tbl.cast(wrong_dtypes_schema), fwrong_types)

In [ ]:
# Write a _common_metadata with the wrong dtypes.
pq.write_metadata(schema=wrong_dtypes_schema, where=output_dataset_path / "_common_metadata")

# Write a _common_metadata with the correct schema but no hats columns.
# This mimics a schema that could have been passed as 'use_schema_file' upon import.
fimport_schema = (output_dataset_path / "_common_metadata").with_suffix(".import_truth")
hats_cols = ["_healpix_29", "Norder", "Dir", "Npix"]
import_schema = pa.schema([fld for fld in input_tbl.schema if fld.name not in hats_cols])
pq.write_metadata(schema=import_schema, where=fimport_schema)

### wrong_files_and_rows

This dataset is designed to fail the following verification tests:

- Files listed in metadata match files on disk.
- Row counts in metadata match row counts on disk and (if provided) user-supplied truth.
- `hats.io.validation.is_valid_catalog`

```
wrong_files_and_rows/
|- properties                               # direct copy of input
|- dataset/
    |- _common_metadata                     # direct copy of input
    |- _metadata                            # missing file 'Npix=11.extra_file.parquet'
    |- Norder=0/Dir=0/
        |- Npix=11.parquet                  # direct copy of input
        |- Npix=11.extra_file.parquet       # added after _metadata generated
        |- Npix=11.extra_rows.parquet       # rows appended after _metadata generated
        |- (Npix=11.missing_file.parquet)   # dropped after _metadata generated
```

In [ ]:
output_dataset_path = Path(".") / "wrong_files_and_rows" / "dataset"
remove_directory(output_dataset_path.parent)

# Existing files may result in unexpected metadata output.
if output_dataset_path.parent.exists() and any(output_dataset_path.parent.iterdir()):
    raise FileExistsError("wrong_files_and_rows directory exists and is not empty. Remove it and try again.")

# We will create the following files using input_frag
ffrag_out = output_dataset_path / frag_key
fmissing_file = ffrag_out.with_suffix(".missing_file.parquet")
fextra_file = ffrag_out.with_suffix(".extra_file.parquet")
fextra_rows = ffrag_out.with_suffix(".extra_rows.parquet")

ffrag_out.parent.mkdir(parents=True, exist_ok=True)

# Copy metadata files that we will not alter
shutil.copy(input_dataset_path.parent / "properties", output_dataset_path.parent / "properties")
shutil.copy(input_dataset_path / "_common_metadata", output_dataset_path / "_common_metadata")

In [ ]:
# Make a direct copy of input_frag for all files that will be recorded in the _metadata file
for file_out in [ffrag_out, fmissing_file, fextra_rows]:
    shutil.copy(input_frag.path, file_out)

# Write _metadata
collect_and_write_metadata(output_dataset_path)

In [ ]:
# Mangle the dataset.

# Add a file
shutil.copy(input_frag.path, fextra_file)

# Remove a file
fmissing_file.unlink()

# Add rows to an existing file
new_tbl = pa.concat_tables([input_tbl, input_tbl.take([1, 2, 3, 4])])
pq.write_table(new_tbl, fextra_rows)

## Create a jagged catalog (to check memory-size partitioning)

Log of strategies tried:
- "Naive uniform" radec across the sphere; arbitrary variation in row sizes
- "Naive uniform" radec across the sphere; exponentially increasing row sizes
- "Naive uniform" radec within a small area; exponentially increasing row sizes
- Don't recall the radec distribution; slightly more even short to long row sizes
- "Naive uniform" radec across the sphere; all positive decs have long rows, neg decs have short
    - This has the problem of pixels at equator containing both long and short, so:
- "Naive uniform" near north pole, then exactly mirrored to near-south-pole; pos decs long and neg decs short

generate dec 70-90
then flip it (mirror the data) - specifically identical, just with a flipped sign for dec

note for uniform on sphere, it'd have to be sin of dec. but it's ok for now as is probably

and then make the pos decs way longer

In [2]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

jagged_path = Path("./small_sky_jagged_source/small_sky_jagged.parquet")

# Delete existing file if it exists
if jagged_path.exists():
    os.remove(jagged_path)

jagged_path.parent.mkdir(parents=True, exist_ok=True)

if not jagged_path.exists():
    num_objects = 2000  # Catalog size
    num_objects_half = num_objects // 2

    # First half: random RA, Dec in [70, 90]
    ra_first = np.random.uniform(0, 360, num_objects_half)
    dec_first = np.random.uniform(70, 90, num_objects_half)

    # Second half: RA identical, Dec sign-flipped
    ra = np.concatenate([ra_first, ra_first])
    dec = np.concatenate([dec_first, -dec_first])

    mag = []
    for i in range(num_objects):
        if i < num_objects_half:
            # Positive Dec: long mag list (e.g., 1000 elements)
            mag.append(np.random.uniform(15, 25, 1000).tolist())
        else:
            # Negative Dec: short mag list (e.g., 5 elements)
            mag.append(np.random.uniform(15, 25, 5).tolist())

    data = {
        "id": np.arange(700, 700 + num_objects),
        "ra": ra,
        "dec": dec,
        "mag": mag,
    }
    df = pd.DataFrame(data)
    df.to_parquet(jagged_path, index=False)
    print(f"Generated simple jagged catalog with long mag lists for positive Dec at {jagged_path}")
else:
    print(f"Jagged test catalog already exists at {jagged_path}")

Generated simple jagged catalog with long mag lists for positive Dec at small_sky_jagged_source/small_sky_jagged.parquet


In [3]:
import pyarrow.parquet as pq
import numpy as np

# Load the jagged catalog
jagged_path = "./small_sky_jagged_source/small_sky_jagged.parquet"
df = pd.read_parquet(jagged_path)

# Compute the size in bytes of each row's 'mag' list (assuming float64)
row_bytes = df["mag"].apply(lambda x: np.array(x, dtype=np.float64).nbytes)
print(f"Maximum bytes in a row: {row_bytes.max()}")
print(f"Row index with max bytes: {row_bytes.idxmax()}")
print(f"Length of 'mag' in that row: {len(df.loc[row_bytes.idxmax(), 'mag'])}")

Maximum bytes in a row: 8000
Row index with max bytes: 0
Length of 'mag' in that row: 1000
